In [1]:
from analyzer.Translator import FaceBookTranslatorProvider
from analyzer.Comparator import PCAComparator
from analyzer.WikiAnalyzer import WikiAnalyzer
from analyzer.EmbeddingService import SentenceTransformerEmbeddingService
from analyzer.ArticleProcessor import ArticleProcessor

import wikipediaapi
import nltk
nltk.download("punkt_tab")  #note: maybe import this in the ArticleProcessor
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
translator_de_en = FaceBookTranslatorProvider("de", "en")
translator_en_de = FaceBookTranslatorProvider("en", "de")

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
sentence_transformer_service = SentenceTransformerEmbeddingService('sentence-transformers/all-mpnet-base-v2')

In [6]:
article_processor = ArticleProcessor("Python (Programmiersprache)", "Python (programming language)", 'de', 'en', 'en', translator_de_en)

In [14]:
comparator = PCAComparator('pairwise', 'article_to_subset', sentence_transformer_service, 0.5, 'sentence-wise', False)

In [15]:
wikiana = WikiAnalyzer(article_processor, comparator)

In [16]:
test = wikiana.analyze()

caching the fileEntwicklungsgeschichte-Zielesubset_to_subset.pkl
caching the fileHistory-Design philosophy and featuressubset_to_subset.pkl


In [17]:
test

[('Indentation', 0.11007854342460632),
 ('Arithmetic operations', 0.11872213333845139),
 ('Methods', 0.12023129314184189)]

In [20]:
# print type of test
type(test)
print(test[0][0])

Indentation


In [5]:
from nltk.corpus import stopwords
import string

def filter_stop_words(text, lang):
    # Get the list of stop words from nltk
    stop_words = set(stopwords.words('english' if lang == 'en' else 'german')) if lang in ['en', 'de'] else set()

    #stop_words = set(stopwords.words('english'))

    # Tokenize the text into words and punctuation
    words = nltk.word_tokenize(text)

    # Filter out the stop words
    filtered_words = [word for word in words if word.lower() not in stop_words or word in string.punctuation]

    # Reconstruct the text
    filtered_text = " ".join(filtered_words)

    # Correct spacing around punctuation
    filtered_text = filtered_text.replace(" ,", ",").replace(" .", ".").replace(" !", "!").replace(" ?", "?")
    filtered_text = filtered_text.replace(" ;", ";").replace(" :", ":").replace(" '", "'")

    return filtered_text

In [13]:
 stop_section_names = ['Einzelnachweise', 'Weiterführende Literatur', 'Notes', 'Quellen',
                         'Weblinks', 'Literatur', 'External links', 'Further reading', 'Sources',
                         'See also', 'References', 'Further reading']

user_agent = 'ba-thesis-comperator (daniel.warkus@hhu.de)'
wiki = wikipediaapi.Wikipedia(user_agent=user_agent , language='de')
page = wiki.page('Python (Programmiersprache)')

sections = {}
for section in page.sections:
   if section.text != '' and section.title not in stop_section_names:
       sections[section.title] = filter_stop_words(section.text, "de")
       for subsection in section.sections:
           if subsection.text != '' and subsection.title not in stop_section_names:
               sections[subsection.title] = filter_stop_words(subsection.text,"de")


In [15]:
import json
with open('data.json', 'w') as f:
    json.dump(sections, f, indent=4)

In [ ]:
# refactored Comperator at line 140 and all other lines like name =... with os.path.join

In [26]:
wiki_wiki = wikipediaapi.Wikipedia('Bachelor-Thesis-Comperator (daniel.warkus@hhu.e)', 'de')
page_py = wiki_wiki.page('Python (Programmiersprache)')

In [29]:
page_py.text

'Python ([ˈpʰaɪθn̩], [ˈpʰaɪθɑn], auf Deutsch auch [ˈpʰyːtɔn]) ist eine universell nutzbare, üblicherweise interpretierte, höhere Programmiersprache. Sie hat den Anspruch, einen gut lesbaren, knappen Programmierstil zu fördern. So werden beispielsweise Blöcke nicht durch geschweifte Klammern, sondern durch Einrückungen strukturiert.\nPython unterstützt mehrere Programmierparadigmen, z. B. die objektorientierte, die aspektorientierte und die funktionale Programmierung. Ferner bietet es eine dynamische Typisierung. Wie viele dynamische Sprachen wird Python oft als Skriptsprache genutzt. Die Sprache weist ein offenes, gemeinschaftsbasiertes Entwicklungsmodell auf, das durch die gemeinnützige Python Software Foundation gestützt wird, die die Definition der Sprache in der Referenzumsetzung CPython pflegt.\n\nEntwicklungsgeschichte\nDie Sprache wurde Anfang der 1990er Jahre von Guido van Rossum am Centrum Wiskunde & Informatica in Amsterdam als Nachfolger für die Programmier-Lehrsprache ABC e

In [34]:
langlinks = page_py.langlinks
en_link = langlinks['en']
# get the english title from the langlinks
en_title = en_link.title
en_title

'Python (programming language)'

In [10]:
# safe the data in a json file
import json
with open('data.json', 'w') as f:
    json.dump(loaded_data, f)

TypeError: Object of type set is not JSON serializable

In [21]:
import Controlling

In [24]:
page_test = Controlling.get_article("Bienen2")

In [25]:
page_test

'No article found'

In [1]:
import sentencepiece


In [2]:
from transformers import pipeline
import torch
translator = pipeline('translation', model='Helsinki-NLP/opus-mt-en-de')
translated_text = translator("This is a sentence to translate from English to German.")
print(translated_text)

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

C:\Users\danie\Documents\Uni\Semester 15\BA\ba-translator\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danie\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-de. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

[{'translation_text': 'Dies ist ein Satz, der vom Englischen ins Deutsche übersetzt werden soll.'}]


In [3]:
type(translated_text)

list

In [4]:
translated_text[0]['translation_text']

'Dies ist ein Satz, der vom Englischen ins Deutsche übersetzt werden soll.'